In [60]:
import numpy as np
from itertools import product,combinations
import matplotlib.pyplot as plt

Problem 1 Pak Kun

In [61]:
epsilon = 1e-2
delta = 0.01
m = 100
gen_max = 300
dim = 2
p_mutation = 0.1
seed = 0

# how many parts/slices do you desire in each dimension?
parts = 100

def system_eq(x):
    f1 = np.exp(x[0]-x[1])-np.sin(x[0]+x[1])
    f2 = (x[0]*x[1])**2-np.cos(x[0]+x[1])
    return np.array([f1,f2])

boundaries = np.array([(-10,10) for _ in range (dim)])

# Objective Function

In [62]:
def objective_function(x:np.ndarray):
    F_array = system_eq(x)
    denom = 0
    for f in F_array:
        denom +=np.abs(f)
    F = 1/(1+denom)
    return -F

In [63]:
def slice_hypercube(lower_bounds, upper_bounds, interval):
    dim = len(lower_bounds)
    # Create a list of arrays, each containing points spaced h apart for each dimension
    points = np.array([np.arange(lower_bounds[i], upper_bounds[i], interval[i]) for i in range(dim)])
    
    # Use meshgrid to create a grid of points in n-dimensional space
    grids = np.array(np.meshgrid(*points, indexing='ij'))
    
    # Flatten and combine the grid points into a single 2D array
    grid_points = np.vstack([grid.ravel() for grid in grids]).T
    
    # Generate all vertices for smaller hypercubes within each grid cell
    offsets = np.array(list(product(*[[0, val] for val in interval])))
    res = np.array([grid_points + offset for offset in offsets])
    return res

In [64]:
# length of each parts in each dimension
inc_int = (boundaries[:,1]-boundaries[:,0])/parts

# Hypercubes
hypercubes_edges = slice_hypercube(lower_bounds=boundaries[:,0],
                                   upper_bounds=boundaries[:,1],
                                   interval=inc_int)
# hypercubes_edges

In [65]:
cluster = []
for hypercube_id in range(hypercubes_edges.shape[1]):
    X0 = hypercubes_edges[:,hypercube_id,:]
    F_list = system_eq(X0.T)

    # cek jika f yang berubah tanda dari F_list jika dievaluasi di tiap edge hypercube
    product_combination = np.array([[a*b for a,b in combinations(F_list[i],2)] for i in range (F_list.shape[0])])

    # jika semua f dari F_list berubah tanda jika dievaluasi di tiap edge hypercube, maka ada akar di situ
    change_sign = np.array([np.any(product_combination[i]<0) for i in range (product_combination.shape[0])])
    if np.all(change_sign==True):
        # print(f'Ada akar di sini: \nX0={X0}')
        cluster.append(X0)

cluster = np.array(cluster)
print(f"Number of clusters containing root: {cluster.shape[0]}")

[[[-6.60000000e+00 -3.55271368e-14]
  [-6.60000000e+00  2.00000000e-01]
  [-6.40000000e+00 -3.55271368e-14]
  [-6.40000000e+00  2.00000000e-01]]

 [[-6.40000000e+00 -2.00000000e-01]
  [-6.40000000e+00 -3.48054918e-14]
  [-6.20000000e+00 -2.00000000e-01]
  [-6.20000000e+00 -3.48054918e-14]]

 [[-6.40000000e+00 -3.55271368e-14]
  [-6.40000000e+00  2.00000000e-01]
  [-6.20000000e+00 -3.55271368e-14]
  [-6.20000000e+00  2.00000000e-01]]

 [[-6.20000000e+00 -2.00000000e-01]
  [-6.20000000e+00 -3.48054918e-14]
  [-6.00000000e+00 -2.00000000e-01]
  [-6.00000000e+00 -3.48054918e-14]]

 [[-1.20000000e+00  1.00000000e+00]
  [-1.20000000e+00  1.20000000e+00]
  [-1.00000000e+00  1.00000000e+00]
  [-1.00000000e+00  1.20000000e+00]]

 [[-1.00000000e+00  1.00000000e+00]
  [-1.00000000e+00  1.20000000e+00]
  [-8.00000000e-01  1.00000000e+00]
  [-8.00000000e-01  1.20000000e+00]]

 [[-2.00000000e-01  6.20000000e+00]
  [-2.00000000e-01  6.40000000e+00]
  [-3.48054918e-14  6.20000000e+00]
  [-3.48054918e-

# GA Evaluation

In [66]:
import importlib
import re

import sys
module_path = r"D:\OneDrive - Institut Teknologi Bandung\[AKADEMIK]\Semester 7-8\TA\Thesis\Genetic Algorithm"
sys.path.append(module_path)
import genal as gal
importlib.reload(gal)

sys.path.remove(module_path)


In [67]:
roots = []
values = []
for i in range (cluster.shape[0]):
    subbound = np.array([[cluster[i,:,:][:,d].min(),cluster[i,:,:][:,d].max()] for d in range(cluster.shape[2])])
    root,value = gal.GA(objective_function=objective_function,
                        population_size=m,
                        boundaries=subbound,
                        max_generation=gen_max,
                        mutation_rate=p_mutation,
                        seed=0,
                        print_stat=True)
    roots.append(root)
    values.append(value)

archive = np.array(roots)
scores = np.array(values)
print(f'Number of Clusters containing root: {cluster.shape[0]}\n')
print(f'Roots:\n{archive}\n\nValues: \n{values}')

=========Generation 0=========
Best Individual: [-6.42066422  0.1565967 ]
Best Score: -0.971961763334752

=========Generation 1=========
Best Individual: [-6.43513451  0.15566445]
Best Score: -0.9942379428230547

=========Generation 2=========
Best Individual: [-6.43513451  0.15566445]
Best Score: -0.9942379428230547

=========Generation 3=========
Best Individual: [-6.43513451  0.15566445]
Best Score: -0.9942379428230547

=========Generation 4=========
Best Individual: [-6.43513451  0.15566445]
Best Score: -0.9942379428230547

=========Generation 5=========
Best Individual: [-6.43780365  0.15566445]
Best Score: -0.9954185120945308

=========Generation 6=========
Best Individual: [-6.43780365  0.15566445]
Best Score: -0.9954185120945308

=========Generation 7=========
Best Individual: [-6.43780365  0.15566445]
Best Score: -0.9954185120945308

=========Generation 8=========
Best Individual: [-6.43780365  0.15566445]
Best Score: -0.9954185120945308

=========Generation 9=========
Best In

c:\Users\ACER\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_qmc.py:763: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)


=========Generation 10=========
Best Individual: [-6.43780365  0.15566445]
Best Score: -0.9954185120945308

=========Generation 11=========
Best Individual: [-6.43780365  0.15566445]
Best Score: -0.9954185120945308

=========Generation 12=========
Best Individual: [-6.43794185  0.15566445]
Best Score: -0.9952392134483176

=========Generation 13=========
Best Individual: [-6.43794185  0.15566445]
Best Score: -0.9952392134483176

=========Generation 14=========
Best Individual: [-6.43794185  0.15566445]
Best Score: -0.9952392134483176

=========Generation 15=========
Best Individual: [-6.43794185  0.15566445]
Best Score: -0.9952392134483176

=========Generation 16=========
Best Individual: [-6.43794185  0.15566445]
Best Score: -0.9952392134483176

=========Generation 17=========
Best Individual: [-6.43794185  0.15566445]
Best Score: -0.9952392134483176

=========Generation 18=========
Best Individual: [-6.43794185  0.15566445]
Best Score: -0.9952392134483176

=========Generation 19======

In [68]:
"""Choosing Best Solution"""
if dim == 1:
    list_criteria = [element for sublist in archive for element in sublist] #convert from 2D array into 1D array
else:
    list_criteria = archive
eligible_roots = np.array([x for x in list_criteria if (objective_function(x))<-1+epsilon])
id_duplicated_roots = []
for i in range(len(eligible_roots)):
    for j in range (i+1,len(eligible_roots)):
        if np.linalg.norm(eligible_roots[i]-eligible_roots[j])<delta:
            id_duplicated_roots.append([i,j])
id_duplicated_roots = np.unique(id_duplicated_roots,axis=0)
deselected_id_duplicated_roots = []
for i in range (len(id_duplicated_roots)):
    root_a = objective_function(eligible_roots[id_duplicated_roots[i][0]])
    root_b = objective_function(eligible_roots[id_duplicated_roots[i][1]])
    if root_a<=root_b:
        id_duplicated_root = id_duplicated_roots[i][1]
    else:
        id_duplicated_root = id_duplicated_roots[i][0]
    deselected_id_duplicated_roots.append(id_duplicated_root)

if deselected_id_duplicated_roots:
    unique_roots = np.ones(len(eligible_roots),dtype=bool)
    unique_roots[deselected_id_duplicated_roots] = False
    final_root = eligible_roots[unique_roots]
else:
    final_root = eligible_roots
print(final_root)

[[-6.43751468  0.15566445]
 [-6.11736654 -0.16334539]
 [-0.93699878  1.06306665]
 [-0.15528234  6.43983731]
 [ 0.16327412  6.12279967]
 [ 0.66733519  0.68990864]]
